In [17]:
import os
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv ,find_dotenv
import openai
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [18]:
load_dotenv(find_dotenv(),override=True)

True

In [19]:
openai.api_key=os.environ["OPENAI_API_KEY"]

### SimpleSequentialChain

In [20]:
from langchain.chains import SequentialChain

In [21]:
llm = OpenAI(temperature=.7)

In [23]:
template = """
Write a blog outline given a topic.
Topic: {topic}
"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
outline_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="outline")

In [24]:
llm = OpenAI(temperature=.7)
template = """Write a blog article based on the below outline.

Outline:
{outline}"""
prompt_template = PromptTemplate(input_variables=["outline"], template=template)
article_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="article")

In [25]:
overall_chain = SequentialChain(
    chains=[outline_chain, article_chain],
    input_variables=["topic"],
    output_variables=["outline", "article"],
    verbose=True)

In [26]:
print(overall_chain({"topic":"Deep Learning"}))



> Entering new SequentialChain chain...

> Finished chain.
{'topic': 'Deep Learning', 'outline': '\n1. Introduction to Deep Learning: Defining and Explaining the Basics \n2. Types of Deep Learning: Supervised, Unsupervised, and Reinforcement \n3. Benefits of Deep Learning: Improved Accuracy and Efficiency \n4. Applications of Deep Learning: Robotics, Autonomous Driving, and Image Recognition \n5. Challenges Faced in Deep Learning: Data Acquisition, Storage, and Labeling \n6. Future of Deep Learning: Advances in AI and Machine Learning \n7. Conclusion: Deep Learning is Here to Stay', 'article': ' \n\nIntroduction to Deep Learning: Defining and Explaining the Basics \n\nDeep learning is an area of artificial intelligence (AI) that utilizes artificial neural networks to enable machines to learn from data without relying on explicitly programmed instructions. It is a subset of machine learning, but it involves more complex data structures and algorithms. Deep learning works by taking inp

In [27]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.llms import OpenAI

In [28]:
llm=OpenAI(temperature=0)
agent_executor=create_python_agent(llm=llm,tool=PythonAstREPLTool(),verbose=True)
agent_executor.run("Calculate the value of 4.9*10 power 4.9 and result would be 2 decimal place")



> Entering new AgentExecutor chain...
 I need to use the python REPL to calculate this
Action: python_repl_ast
Action Input: 4.9*10**4.9
Observation: 389220.8350148983
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


 I need to round the result to two decimal places
Action: python_repl_ast
Action Input: round(389220.8350148983, 2)
Observation: 389220.84
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals

 I now know the final answer
Final Answer: 389220.84

> Finished chain.


'389220.84'

### exampl 2

In [29]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)
# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

In [30]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [31]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [32]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
	agent="zero-shot-react-description",
	tools=tools,
	llm=llm,
	verbose=True,
	max_iterations=3
)

In [33]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals


Observation: Answer: 139.94261298333066
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals

 I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [34]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")
     



> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals

 I need to figure out how many apples are in the boxes
Action: Calculator
Action Input: 8 * 2.5

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..



Observation: Answer: 20.0
Thought:

...

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals


Observation: Answer: 24.0
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals

 I now know the final answer
Final Answer: We have 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'We have 24 apples.'}

In [35]:
zero_shot_agent("what is the capital of India?")



> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals

 I need to look up the answer
Action: Look up
Action Input: Capital of India
Observation: Look up is not a valid tool, try one of [Calculator].
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals

 I need to use the calculator
Action: Calculator
Action Input: Capital of India

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals

ValueError: unknown format from LLM: This question cannot be answered using the numexpr library, as it is not a mathematical expression.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{ dcvfb}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

In [ ]:
tools.append(llm_tool)

In [ ]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [ ]:
zero_shot_agent("what is the capital of Norway?")
     

In [ ]:
zero_shot_agent("what is (4.5*2.1)^2.2?")